In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [2]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')

In [3]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [4]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'
# sheet_id = '1XqOtPkiE_Q0dfGSoyxrH730RkwrTczcRbDeJJpqRByQ'
sheet_name = 'quick'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=quick'

In [5]:
output_file_path = '../data/labels/gt-bands.xlsx'

In [6]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,Notes on how these were chosen,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,quick-0,30.234561,77.949155,gravel,2020-05-31,fluvial,https://doi.org/10.1130/B35334.1,India,Mohand river,Within Siwalik conglomerate ~10 km upstream of...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,quick-1,30.221258,77.938005,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.2,India,Mohand river,Within Siwalik conglomerate ~7 km upstream of MFT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,quick-2,30.195490,77.913587,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.3,India,Mohand river,Within Middle Siwalik sandstone ~3 km upstream...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,quick-3,30.175600,77.902492,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.4,India,Mohand river,Mountain front (MFT) Near Mohand town,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,quick-4,30.141309,77.876064,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.5,India,Mohand river,~5 km downstream of MFT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,quick-5,30.104132,77.862792,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.6,India,Mohand river,~10 downstream of MFT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,quick-6,30.085487,77.848255,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.7,India,Mohand river,~13 km downstream of MFT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,quick-7,30.234482,77.948796,gravel,2021-02-10,fluvial,https://doi.org/10.1130/B35334.8,India,Mohand river,Within Siwalik conglomerate ~10 km upstream of...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,quick-8,28.544683,81.186374,gravel,2020-12-05,fluvial,https://doi.org/10.1130/B35334.9,Nepal,Karnali river,~15 km downstream of MFT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,quick-9,28.467970,81.111664,gravel,2021-01-06,fluvial,https://doi.org/10.1130/B35334.10,Nepal,Karnali river,~28 km downstream of MFT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
#drop any columns that have 'Unnamed:' in the column name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,Notes on how these were chosen
0,quick-0,30.234561,77.949155,gravel,2020-05-31,fluvial,https://doi.org/10.1130/B35334.1,India,Mohand river,Within Siwalik conglomerate ~10 km upstream of...
1,quick-1,30.221258,77.938005,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.2,India,Mohand river,Within Siwalik conglomerate ~7 km upstream of MFT
2,quick-2,30.195490,77.913587,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.3,India,Mohand river,Within Middle Siwalik sandstone ~3 km upstream...
3,quick-3,30.175600,77.902492,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.4,India,Mohand river,Mountain front (MFT) Near Mohand town
4,quick-4,30.141309,77.876064,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.5,India,Mohand river,~5 km downstream of MFT
5,quick-5,30.104132,77.862792,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.6,India,Mohand river,~10 downstream of MFT
6,quick-6,30.085487,77.848255,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.7,India,Mohand river,~13 km downstream of MFT
7,quick-7,30.234482,77.948796,gravel,2021-02-10,fluvial,https://doi.org/10.1130/B35334.8,India,Mohand river,Within Siwalik conglomerate ~10 km upstream of...
8,quick-8,28.544683,81.186374,gravel,2020-12-05,fluvial,https://doi.org/10.1130/B35334.9,Nepal,Karnali river,~15 km downstream of MFT
9,quick-9,28.467970,81.111664,gravel,2021-01-06,fluvial,https://doi.org/10.1130/B35334.10,Nepal,Karnali river,~28 km downstream of MFT


In [8]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [9]:
df.columns

Index(['ID', 'Latitude', 'Longitude', 'Class', 'Date', 'Site', 'Source',
       'Country', 'Notes', 'Notes on how these were chosen'],
      dtype='object')

In [10]:
ee.Initialize()
# ee.Authenticate()

In [11]:
output = df.copy()
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,Notes on how these were chosen
0,quick-0,30.234561,77.949155,gravel,2020-05-31,fluvial,https://doi.org/10.1130/B35334.1,India,Mohand river,Within Siwalik conglomerate ~10 km upstream of...
1,quick-1,30.221258,77.938005,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.2,India,Mohand river,Within Siwalik conglomerate ~7 km upstream of MFT
2,quick-2,30.195490,77.913587,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.3,India,Mohand river,Within Middle Siwalik sandstone ~3 km upstream...
3,quick-3,30.175600,77.902492,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.4,India,Mohand river,Mountain front (MFT) Near Mohand town
4,quick-4,30.141309,77.876064,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.5,India,Mohand river,~5 km downstream of MFT
5,quick-5,30.104132,77.862792,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.6,India,Mohand river,~10 downstream of MFT
6,quick-6,30.085487,77.848255,gravel,2021-12-07,fluvial,https://doi.org/10.1130/B35334.7,India,Mohand river,~13 km downstream of MFT
7,quick-7,30.234482,77.948796,gravel,2021-02-10,fluvial,https://doi.org/10.1130/B35334.8,India,Mohand river,Within Siwalik conglomerate ~10 km upstream of...
8,quick-8,28.544683,81.186374,gravel,2020-12-05,fluvial,https://doi.org/10.1130/B35334.9,Nepal,Karnali river,~15 km downstream of MFT
9,quick-9,28.467970,81.111664,gravel,2021-01-06,fluvial,https://doi.org/10.1130/B35334.10,Nepal,Karnali river,~28 km downstream of MFT


In [12]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm
import file_management as fm

In [34]:
# INDEX = 5
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [38]:
s1_s2, sample = qm.get_s1s2_data(output, Map, INDEX, display_smap=True, mosaic_method='qm', sampling_buffer_m=sampling_buffer_m, median_samples=5,roi_buffer_m=1200, obia=True)

Index:  13  ID:  quick-13 Class:  gravel  Site:  fluvial
Search window from 2020-06-23 to 2021-06-23


Map(center=[26.726536007916483, 87.1084140765723], controls=(ZoomControl(options=['position', 'zoom_in_text', …

## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function

In [39]:
output, INDEX = qm.get_training_sample(output, s1_s2, sample, Map, INDEX, display_clusters=False, obia=True)
# fm.write_to_excel(output, output_file_path, sheet_name)
with pd.ExcelWriter(output_file_path,engine="openpyxl",mode="a",if_sheet_exists="replace") as writer:
    output.to_excel(writer,sheet_name=sheet_name,index=False)
output.iloc[INDEX-1]

Discarded Observation


ID                                                                         quick-13
Latitude                                                                  26.726536
Longitude                                                                 87.108414
Class                                                                        gravel
Date                                                                     2020-12-23
Site                                                                        fluvial
Source                                            https://doi.org/10.1130/B35334.14
Country                                                                       Nepal
Notes                                                                    Kosi river
Notes on how these were chosen    ~15 km downstream of MFT,   Patchy gravel bar/...
B2_mean                                                                         NaN
B3_mean                                                                     

## Final Output

In [21]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2_mean,...,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked,class_code
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for,1024.304688,...,3794.05957,3441.761475,-13.875913,-24.152828,0.183746,0.244367,-0.246776,True,False,1
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
